In [ ]:
import pandas as pd
import folium
from folium import IFrame
import matplotlib.pyplot as plt
from io import BytesIO
import base64

# Load data
header_path = "C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/ProcessedPAGData/PAG_BinHeader_mapped.csv"
header_data = pd.read_csv(header_path)

count_file_path = "C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/ProcessedPAGData/most_recent_BinCount.csv"
count_data = pd.read_csv(count_file_path)

location_file_path = "C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/ProcessedPAGData/PAG_locations_2023.csv"
location_data = pd.read_csv(location_file_path)

# Convert count data based on header mapping
standard_classes = ['Motorcycle', 'Short_vehicle', 'Medium_vehicle', 'Long_vehicle']
conversion_dict = header_data.set_index('ID')[['Head' + str(i) for i in range(1, 16)]].to_dict('index')

for idx, row in count_data.iterrows():
    id_conversions = conversion_dict.get(row['ID'], {})
    for cls in standard_classes:
        count_data.loc[idx, cls] = sum(row['Class' + str(i)] for i, head in enumerate(id_conversions.values(), 1) if head == cls)

# Filter and merge with location data
coordinates = pd.merge(header_data[['LOCAL_ID', 'ID']], location_data[['LOCAL_ID', 'LATITUDE', 'LONGITUDE']], on='LOCAL_ID', how='left')
merged_data = pd.merge(coordinates, count_data[['ID'] + standard_classes], on='ID', how='left')

# Create a folium map centered on Tucson, AZ
map_location = [32.2226, -110.9747]
m = folium.Map(location=map_location, zoom_start=12, tiles='cartodbpositron')

# Define colors for each vehicle class
class_colors = {
    'Motorcycle': '#000000',  
    'Short_vehicle': '#3F80F9', 
    'Medium_vehicle': '#FF4747',  
    'Long_vehicle': '#7AF353'  
}

# Function to create pie charts as base64 images
def create_pie_chart(ratios, labels):
    fig, ax = plt.subplots(figsize=(3.5, 3.5), dpi=100)  # Increase size
    colors = [class_colors[label] for label in labels]
    ax.pie(ratios, colors=colors, startangle=90)
    ax.axis('equal')
    plt.close(fig)
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', transparent=True)
    buf.seek(0)
    return f"data:image/png;base64,{base64.b64encode(buf.read()).decode('utf8')}"

# Add pie charts to the map with popups
for index, row in merged_data.iterrows():
    ratios = [row[cls] for cls in standard_classes if row[cls] > 0]
    labels = [cls for cls in standard_classes if row[cls] > 0]
    total = sum(ratios)
    if total > 0:
        xy = (row['LATITUDE'], row['LONGITUDE'])
        if pd.notna(xy[0]) and pd.notna(xy[1]):
            popup_text = f"ID: {row['ID']}<br>" + "<br>".join(
                f"{cls}: {int(row[cls])}" for cls in labels
            )
            marker = folium.Marker(
                location=xy,
                popup=folium.Popup(html=popup_text, max_width=400),
                icon=folium.DivIcon(html=f'<div style="width: 80px; height: 80px;"><img src="{create_pie_chart(ratios, labels)}" style="width:80px;height:80px;"></div>')  # Increased size
            )
            marker.add_to(m)

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 140px; 
     border:2px solid grey; z-index:9999; font-size:14px; background-color: white;
     ">&nbsp; <b>Vehicle Classes</b> <br>
     &nbsp; Motorcycle: <i style="background:#000000;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Short_vehicle: <i style="background:#3F80F9;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Medium_vehicle: <i style="background:#FF4747;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Long_vehicle: <i style="background:#7AF353;width:10px;height:10px;display:inline-block"></i>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Save the results
map_output_path = "C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/results/pie_map/interactive_pie_chart_map_static.html"
m.save(map_output_path)
print(f"Map saved to {map_output_path}")

C:\Users\yejijeon\AppData\Local\Temp\ipykernel_8812\1143370511.py:10: DtypeWarning: Columns (10,15,20,21,22,23,24,25,26,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  header_data = pd.read_csv(header_path)


Map saved to C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/results/pie_map/interactive_pie_chart_map_static.html


In [ ]:
import pandas as pd
import folium
from matplotlib import pyplot as plt
from io import BytesIO
import base64

# Load data
intersection_data = pd.read_csv("C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/20240401_PAG_TMC_Backup/TmcIntersection.CSV")
count_data = pd.read_csv("C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/20240401_PAG_TMC_Backup/TmcCount.CSV")
count_data['StartDate'] = (count_data['StartDate'] / 1e7).astype(int).astype(str)
count_data['StartDate'] = pd.to_datetime(count_data['StartDate'], format='%Y%m%d%H%M%S', errors='coerce')
count_data['Year'] = count_data['StartDate'].dt.year
count_data.dropna(subset=['IntersectionId', 'Year'], inplace=True)
count_data['IntersectionId'] = count_data['IntersectionId'].astype(int)

# Only keep the most recent counts for each IntersectionId
most_recent_counts = count_data.loc[count_data.groupby('IntersectionId')['StartDate'].idxmax()]

# Initialize map
m = folium.Map(location=[32.2226, -110.9747], zoom_start=10, tiles='cartodbpositron')

# Colors and names for classes
class_colors = {
    1: '#89d1ff',  # Car or class 1
    2: '#9a9a9a',  # Truck or class 2
    3: '#ff8b8b',  # Bike or class 3
    4: '#a7ff94'   # Pedestrian or class 4
}
class_names = {
    1: 'Car',
    2: 'Truck',
    3: 'Bike',
    4: 'Pedestrian'
}

# Function to create pie charts as base64 images
def create_pie_chart(ratios, labels):
    fig, ax = plt.subplots(figsize=(1.5, 1.5), dpi=100)
    colors = [class_colors.get(label, '#808080') for label in labels]
    ax.pie(ratios, colors=colors, startangle=90)
    ax.axis('equal')
    plt.close(fig)
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', transparent=True)
    buf.seek(0)
    return f"data:image/png;base64,{base64.b64encode(buf.read()).decode('utf8')}"

# Process data and add pie charts to map
for chunk in pd.read_csv("C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/data/20240401_PAG_TMC_Backup/TmcCountData.CSV", chunksize=50000):
    chunk = chunk.merge(most_recent_counts, on='CountId', how='inner')
    chunk = chunk.merge(intersection_data, on='IntersectionId', how='inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 'HardRightMovement', 'UTurnMovement']].sum(axis=1)
    chunk = chunk[chunk['ClassificationId'] <= 4]
    classification_summary = chunk.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 'Longitude', 'StartDate'])['TotalCount'].sum().reset_index()
    
    for intersection_id, group in classification_summary.groupby('IntersectionId'):
        labels = group['ClassificationId'].tolist()
        values = group['TotalCount'].tolist()
        if labels == [3]:
            continue
        popup_text = f"<b>Intersection ID:</b> {intersection_id}<br>"
        popup_text += f"<b>Date:</b> {group['StartDate'].iloc[0].strftime('%Y-%m-%d')}<br>"
        for label, value in zip(labels, values):
            class_name = class_names.get(label, f"Class {label}")
            popup_text += f"<b>{class_name}:</b> {value}<br>"
        if sum(values) > 0:
            pie_chart = create_pie_chart(values, labels)
            folium.Marker(
                location=[group['Latitude'].iloc[0], group['Longitude'].iloc[0]],
                popup=folium.Popup(html=popup_text, max_width=300),
                icon=folium.DivIcon(html=f'''
                    <div id="marker-{intersection_id}" style="width:40px;height:40px;">
                        <img src="{pie_chart}" style="width:100%;height:100%;">
                    </div>
                ''')
            ).add_to(m)

# Add a legend to the map
legend_html = '''
<div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 140px; 
     border:2px solid grey; z-index:9999; font-size:14px; background-color: white;
     ">&nbsp; <b>Classification Types</b> <br>
     &nbsp; Car: <i style="background:#89d1ff;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Truck: <i style="background:#9a9a9a;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Bike: <i style="background:#ff8b8b;width:10px;height:10px;display:inline-block"></i><br>
     &nbsp; Pedestrian: <i style="background:#a7ff94;width:10px;height:10px;display:inline-block"></i>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

m.save("C:/Users/yejijeon/OneDrive/PAG project/Enhance Traffic Count/results/pie_map/multimodal_pie_chart_map_dynamic.html")
print("Map saved successfully")

Map saved successfully
